<a href="https://colab.research.google.com/github/royam0820/prompt-engineering/blob/main/l8_chatbot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.


## Setup

In [1]:
#!pip install openai==0.27
!pip install openai

In [2]:
import openai
import os
from google.colab import userdata

In [3]:
from pprint import pprint

In [4]:
print(openai.__version__)

1.35.3


In [5]:
openai.api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [6]:
def get_completion(prompt, model="gpt-3.5-turbo",temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,   # this is the degree of randomness of the model's output
    )
    # Access the content directly from the 'message' attribute
    return response.choices[0].message.content

In [7]:
# initializing the OpenAI client
client = openai.OpenAI()

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

NB: The line `client=openai.OpenAI()` initializes the OpenAI client.  It typically means creating an instance of a class that handles the details of communicating with the API. However, the OpenAI Python client library simplifies this by allowing direct calls without explicitly initializing a client object. If your setup requires this line, it indicates that the client object is used to handle API interactions. This isn't standard for most setups, so check your environment or any custom configurations.

In [8]:
# client = openai.OpenAI()

# def get_completion_from_messages(messages,
#                                  model="gpt-3.5-turbo",
#                                  temperature=0,
#                                  max_tokens=500):
#     response = client.chat.completions.create(model=model,
#     messages=messages,
#     temperature=temperature,
#     max_tokens=max_tokens)
#     return response.choices[0].message.content



NB: with the above commented code, you can a `max_tokens` parameter. It is often useful to include it to control the length of the response and avoid excessive usage of tokens which could result in higher costs or unexpected behavior.

In [8]:
# initialising messages
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

NB: In a conversational AI system, the roles "system," "user," and "assistant" define the context and interaction flow. Here’s an explanation of each role based on the provided code:
- **The system role**: The "system" role sets the initial instructions or context for the assistant. It helps to define the behavior, style, or rules that the assistant should follow throughout the conversation.
- **The user role**: The "user" role represents the inputs or queries made by the person interacting with the assistant. These inputs drive the conversation and determine the assistant's subsequent responses.
- **The assistant role**: The "assistant" role represents the AI or conversational agent's responses to the user's inputs. The responses are based on the instructions given by the "system" and the inputs from the "user."

In [10]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Verily, the fowl crossed the thoroughfare to reach the other side, forsooth!


NB: Shakespearean answer! In plain English, it will be: "To get to the other side! It's a joke that always makes people laugh."

The joke "Why did the chicken cross the road? To get to the other side!" is a classic example of an anti-joke. An anti-joke is a type of humor where the punchline is deliberately mundane or straightforward rather than a typical witty or humorous twist. The humor comes from subverting the listener's expectations.

"L'énigme 'Pourquoi le poulet a-t-il traversé la route ? Pour aller de l'autre côté !' est un exemple classique de blague anti-humoristique. Une blague anti-humoristique est un type d'humour où la chute est délibérément banale ou directe plutôt que spirituelle ou humoristique. L'humour vient de la subversion des attentes de l'auditeur."

In [10]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1) # higher temperature
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?


NB: Asssitant response.

In [11]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as a chatbot, I do not have the ability to know your name. Feel free to introduce yourself if you'd like!


NB: The model does not remember the name of the user.  Each conversaion with a language model is a standalone interaction which means that you must provide all relevant messages for the model to draw from in the current conversation. To have the model remember past messages, you must provide them, see below.

In [12]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa. It's a pleasure to chat with you!


NB: the model was able to respond because it has all the context it needs to recall the user's name.

# Gradio Chatbot (new amr)

In [15]:
!pip install gradio

In [ ]:
import gradio as gr
import openai
import os

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = openai.OpenAI()

# Initialize conversation history
conversation_history = [
    {'role':'system', 'content':"""
    You are OrderBot, an automated service to collect orders for a pizza restaurant.
    You first greet the customer, then collects the order,
    and then asks if it's a pickup or delivery.
    You wait to collect the entire order, then summarize it and check for a final
    time if the customer wants to add anything else.
    If it's a delivery, you ask for an address.
    Finally you collect the payment.
    Make sure to clarify all options, extras and sizes to uniquely
    identify the item from the menu.
    You respond in a short, very conversational friendly style.
    The menu includes
    pepperoni pizza 12.95, 10.00, 7.00
    cheese pizza 10.95, 9.25, 6.50
    eggplant pizza 11.95, 9.75, 6.75
    fries 4.50, 3.50
    greek salad 7.25
    Toppings:
    extra cheese 2.00,
    mushrooms 1.50
    sausage 3.00
    canadian bacon 3.50
    AI sauce 1.50
    peppers 1.00
    Drinks:
    coke 3.00, 2.00, 1.00
    sprite 3.00, 2.00, 1.00
    bottled water 5.00
    """}
]

def chatbot(message, history):
    global conversation_history

    # Add user message to conversation history
    conversation_history.append({"role": "user", "content": message})

    # Get response from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    ai_message = response.choices[0].message.content

    # Add AI response to conversation history
    conversation_history.append({"role": "assistant", "content": ai_message})

    # Return the full conversation history
    return [(conversation_history[i]["content"], conversation_history[i+1]["content"])
            for i in range(1, len(conversation_history)-1, 2)]

# Function to initialize the chat with a greeting
def init_chat():
    global conversation_history

    # Get initial greeting from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    greeting = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": greeting})

    return [("", greeting)]

iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(placeholder="Type your message here..."),
    outputs=gr.Chatbot(height=500),
    title="Pizza OrderBot",
    description="I'm here to help you order pizza. What would you like?",
    allow_flagging="never",
)

# Launch the interface with the initial greeting
iface.launch(lambda: init_chat())#

In [ ]:
# this code displayed the whole message conversation in a JSON format
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "cheese pizza",
    "size": "large",
    "price": 10.95
  },
  "toppings": [],
  "drinks": [],
  "sides": [],
  "total price": 10.95
}


# Panel OrderBot (orig)
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [11]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


NB: The code above ia an helper function, we will collect all the conversations beween the user and the model (the assistant) from the user interface.

The function `collect_messages` captures user input, updates the conversation context, generates a response from the assistant, and updates the UI to reflect the conversation. The UI components are managed using `pn` (presumably panel library) for creating interactive web applications.

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [14]:
# this code displayed the whole message conversation
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "cheese pizza",
    "size": "large",
    "price": 10.95
  },
  "toppings": [],
  "drinks": [],
  "sides": [],
  "total price": 10.95
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

### Experimenting on role attribution

In [13]:
# system role example
{'role': 'system', 'content': 'You are an assistant that speaks like Shakespeare.'}

{'role': 'system',
 'content': 'You are an assistant that speaks like Shakespeare.'}

NB: This instructs the assistant to adopt a Shakespearean style in its responses.

In [15]:
# user role example
{'role': 'user', 'content': 'tell me a joke'}
{'role': 'user', 'content': 'I don\'t know'}


{'role': 'user', 'content': "I don't know"}

NB: These show the user asking for a joke and then responding to the assistant's joke setup

In [17]:
# assitant role exemple
{'role': 'assistant', 'content': 'Why did the chicken cross the road'}


{'role': 'assistant', 'content': 'Why did the chicken cross the road'}

NB: Here, the assistant responds to the user's request for a joke by starting a classic joke.

These roles collectively structure the dialogue, enabling the assistant to understand its behavior (as dictated by the system), respond to user queries, and maintain a coherent conversation flow.

### My Travel Chatbot

In [ ]:
import gradio as gr
import openai
import os

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = openai.OpenAI()

# Initialize conversation history
conversation_history = [
    {'role':'system', 'content':"""
    You are TravelBot, an automated travel planning assistant.
    You help users plan their vacations by suggesting destinations, activities, and accommodations.
    First, greet the user and ask about their travel preferences (e.g., budget, desired climate, type of trip).
    Then, suggest destinations based on their preferences.
    Once a destination is chosen, recommend activities and attractions.
    Ask about the length of stay and suggest appropriate accommodations.
    Finally, summarize the travel plan and ask if they want to make any changes.
    You respond in a friendly, enthusiastic, and informative style.
    You have knowledge of popular destinations worldwide, including:

    Beach destinations: Bali, Hawaii, Maldives, Caribbean Islands
    City breaks: Paris, New York, Tokyo, London
    Nature and adventure: Costa Rica, New Zealand, Switzerland, Canada
    Cultural experiences: Italy, Japan, India, Egypt

    For each destination, you can suggest:
    - Popular attractions and activities
    - Typical weather and best times to visit
    - Accommodation options (budget hostels to luxury resorts)
    - Local cuisine and dining recommendations
    - Transportation options
    - Estimated costs for various aspects of the trip
    """
    }
]

def chatbot(message, history):
    global conversation_history

    # Add user message to conversation history
    conversation_history.append({"role": "user", "content": message})

    # Get response from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    ai_message = response.choices[0].message.content

    # Add AI response to conversation history
    conversation_history.append({"role": "assistant", "content": ai_message})

    # Return the full conversation history
    return [(conversation_history[i]["content"], conversation_history[i+1]["content"])
            for i in range(1, len(conversation_history)-1, 2)]

# Function to initialize the chat with a greeting
def init_chat():
    global conversation_history

    # Get initial greeting from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    greeting = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": greeting})

    return [("", greeting)]

iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(placeholder="Type your travel preferences or questions here..."),
    outputs=gr.Chatbot(height=500),
    title="TravelBot - Your Personal Travel Planner",
    description="I'm here to help you plan your perfect vacation. What kind of trip are you looking for?",
    allow_flagging="never",
)

# Launch the interface with the initial greeting
iface.launch(lambda: init_chat())

In [19]:
import gradio as gr
import openai
import os

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
client = openai.OpenAI()

# Initialize conversation history
conversation_history = [
    {'role':'system', 'content':"""
    You are TravelBot, an automated travel planning assistant.
    You help users plan their vacations by suggesting destinations, activities, and accommodations.
    First, greet the user and ask about their travel preferences (e.g., budget, desired climate, type of trip).
    Then, suggest destinations based on their preferences.
    Once a destination is chosen, recommend activities and attractions.
    Ask about the length of stay and suggest appropriate accommodations.
    Finally, summarize the travel plan and ask if they want to make any changes.
    You respond in a friendly, enthusiastic, and informative style.
    You have knowledge of popular destinations worldwide, including:

    Beach destinations: Bali, Hawaii, Maldives, Caribbean Islands
    City breaks: Paris, New York, Tokyo, London
    Nature and adventure: Costa Rica, New Zealand, Switzerland, Canada
    Cultural experiences: Italy, Japan, India, Egypt

    For each destination, you can suggest:
    - Popular attractions and activities
    - Typical weather and best times to visit
    - Accommodation options (budget hostels to luxury resorts)
    - Local cuisine and dining recommendations
    - Transportation options
    - Estimated costs for various aspects of the trip
    """
    }
]

def chatbot(message, history):
    global conversation_history

    # Add user message to conversation history
    conversation_history.append({"role": "user", "content": message})

    # Get response from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    ai_message = response.choices[0].message.content

    # Add AI response to conversation history
    conversation_history.append({"role": "assistant", "content": ai_message})

    # Return the last exchange
    return ai_message

# Function to initialize the chat with a greeting
def init_chat():
    global conversation_history

    # Get initial greeting from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    greeting = response.choices[0].message.content
    conversation_history.append({"role": "assistant", "content": greeting})

    return [("", greeting)]

# Function to clear the input field
def clear_input():
    return ""

with gr.Blocks() as iface:
    chatbot_component = gr.Chatbot(height=500)
    msg = gr.Textbox(placeholder="Type your travel preferences or questions here...")
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_message = chatbot(user_message, history)
        history[-1][1] = bot_message
        return history

    msg.submit(user, [msg, chatbot_component], [msg, chatbot_component]).then(
        bot, chatbot_component, chatbot_component
    )
    clear.click(clear_input, outputs=msg)

    iface.load(init_chat, outputs=chatbot_component)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e084ad90b53920cc3b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Displaying a complete summary of the travel bot conversation.

In [22]:
import json

def get_travel_summary():
    global conversation_history

    # Add a system message to request a JSON summary
    conversation_history.append(
        {'role':'system', 'content':'''
        Create a JSON summary of the travel plan discussed. Include the following fields:
        1) destination
        2) duration (in days)
        3) accommodations (list with prices if mentioned)
        4) activities (list with prices if mentioned)
        5) transportation (list with prices if mentioned)
        6) estimated total budget

        If any information is not available, use "Not specified" as the value.
        '''}
    )

    # Get response from OpenAI
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    summary = response.choices[0].message.content

    # Remove the summary request from conversation history
    conversation_history.pop()

    try:
        # Parse the JSON response
        summary_json = json.loads(summary)
        return summary_json
    except json.JSONDecodeError:
        return {"error": "Failed to generate a valid JSON summary"}

# Add this function to your Gradio interface
def display_summary():
    summary = get_travel_summary()
    if "error" in summary:
        return summary["error"]
    else:
        return json.dumps(summary, indent=2)

# Modify your Gradio interface to include a summary button
with gr.Blocks() as iface:
    chatbot_component = gr.Chatbot(height=500)
    msg = gr.Textbox(placeholder="Type your travel preferences or questions here...")
    clear = gr.Button("Clear")
    summary_button = gr.Button("Get Travel Summary")
    summary_output = gr.Textbox(label="Travel Summary", lines=10)

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_message = chatbot(user_message, history)
        history[-1][1] = bot_message
        return history

    msg.submit(user, [msg, chatbot_component], [msg, chatbot_component]).then(
        bot, chatbot_component, chatbot_component
    )
    clear.click(clear_input, outputs=msg)
    summary_button.click(display_summary, outputs=summary_output)

    iface.load(init_chat, outputs=chatbot_component)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3f72b41a57817050c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
